In [ ]:
using ForwardDiff
using GynC
using Plots; pyplot()
using StatPlots

In [ ]:
using Parameters

@with_kw struct Model
  f = x->x.^2
  n = 100
  xs = linspace(1,2,n)
  σ = 1
  γ = 1
end

function generatedata(m, n; prior=dataprior)
  # smooth, true samples
  #dataxs = rand(prior, n)
  # discrete samples
  dataxs = m.xs[rand(Categorical(wtrue), n)]
  datays = m.f.(dataxs) + rand(Normal(0, m.σ), n)
  dataxs, datays
end

jeffreysprior(m) = normalize([ForwardDiff.derivative(m.f, x) for x in m.xs], 1)
  
likelihoodmat(m, data) = [pdf(Normal(y, m.σ), d) for d in data, y in m.f.(m.xs)]


logL(w, L)  = sum(log.(L * w)) 
dlogL(w, L) = sum(L ./ (L*w), 1) |> vec

dklj(w, j)  = sum(w[i] * log(w[i]/j[i]) for i in 1:length(w))
ddklj(w, j) = [log(w[i]/j[i]) + 1 for i in 1:length(w)]

function ebprior(m, datas, γ, c=GynC.OptConfig(n=MAXEVAL))
  
  data = datas[2]
  
  j = jeffreysprior(m)
  L = likelihoodmat(m, data)  
  
   obj(w) = (γ > 0 ? γ *  dklj(w,j) : 0) -  logL(w,L)
  dobj(w) = (γ > 0 ? γ * ddklj(w,j) : 0) - dlogL(w,L)
  
  opt = GynC.simplex_minimize(obj, dobj, ones(m.n), config=c)
  global opthist 
  opthist = opt[2]
  opt[1] 
end

m = Model()
dataprior = TruncatedNormal(1.5, 0.1, 1, 2)
wtrue = normalize(pdf.(dataprior, m.xs), 1);


In [ ]:
priorpredictivedistr(m, w) = MixtureModel([Normal(m.f(x), m.σ) for x in m.xs], w)
plotpriorpredictivedistr!(m, w) = plot!(0:0.01:5, x->pdf(priorpredictivedistr(m,w),x))

# Single run

- peaking for fine discretization
- ilja conj: vanished for high M

In [ ]:
srand(1)
datas = generatedata(m, 100);
c = GynC.OptConfig(XTOLABS=1e-5, OPTIMIZER=:LD_MMA)
@time w = ebprior(m, datas, 0, c)

@show length(opthist)

@show logL(w, L)
@show logL(w, L) - logL(wtrue, L)
println()

@show norm(w-wtrue)
@show maximum(w)

plot(m.xs, w*m.n)
plot!(dataprior)

In [ ]:
plot()
plotpriorpredictivedistr!(m,w)
plotpriorpredictivedistr!(m,wtrue)

# $M \rightarrow \infty$

- error increases

In [ ]:
function exp_ndata(ndata::Int, optc)
  m = Model()
  datas = generatedata(m, ndata)
  w = ebprior(m, datas, 0, optc)
  global opthist
  w, opthist, norm(w-wtrue)/m.n, datas
end

c = GynC.OptConfig(XTOLABS=1e-5)
ndatarange = round.(Int,logspace(0,4,12))
t = @elapsed(exps = map(n->exp_ndata(n,c), ndatarange));

In [ ]:
@show t;
plot(ndatarange, map(r->length(r[2]), exps),xaxis=:log10) |> display

ws = getindex.(exps,[1])
plot(ndatarange, map(w->norm(w - wtrue), ws),xaxis=:log)

# Convergence for different $\gamma$

In [ ]:
err(w) = norm(w - wtrue)

gammas = [0,1,10,100,1000,10000]
ndatas = 10.^collect(0:6)
iters = 10
c = GynC.OptConfig(FTOLREL=1e-6)

ps = []
@time for gamma in gammas 
  plot()
  i=0
  for ndata in ndatas
    i+=1
    errs = [err(ebprior(m, generatedata(m, ndata), gamma, c)) for i = 1:iters]
    boxplot!([i],errs)
  end
  
  push!(ps, plot!(legend=false, title=gamma, ylims=[0,.35]))
end

In [ ]:
pp = plot(ps...)